# Wildlife Camera Trap Classification with Azure Custom Vision
*A Microsoft AI Services Tutorial for Ecological Research*

## What This Demonstrates

**Microsoft Custom Vision** for automated wildlife monitoring research. We'll show how to:
- Set up an Azure Custom Vision project 
- Train a model with minimal wildlife data
- Use the prediction API for batch processing
- Export results for research analysis

**Why Custom Vision for Research:**
- **No ML expertise required** - visual training interface
- **Small data friendly** - works with as few as 5 images per category  
- **Research-ready APIs** - confidence scores and batch processing
- **Scalable** - handles thousands of images via REST API

## Setup: Create Azure Custom Vision Resource

### Step 1: Azure Portal Setup
**In Azure Portal:**
1. Go to **Create a resource** → **AI + Machine Learning** → **Custom Vision**
2. Create **Training resource** (F0 free tier available)
3. Create **Prediction resource** (F0 free tier available)  
4. Note down: **Keys**, **Endpoints**, and **Resource IDs**

In [ ]:
# Install required packages
!pip install azure-cognitiveservices-vision-customvision
!pip install azure-cognitiveservices-vision-computervision  
!pip install pillow matplotlib pandas requests

In [ ]:
# Import libraries
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry
from msrest.authentication import ApiKeyCredentials
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import requests
import time
import io

## Azure Custom Vision Configuration

In [ ]:
# Azure Custom Vision credentials (replace with your actual values)
ENDPOINT = "https://your-resource.cognitiveservices.azure.com/"
TRAINING_KEY = "your_training_key_here"
PREDICTION_KEY = "your_prediction_key_here" 
PREDICTION_RESOURCE_ID = "/subscriptions/your-sub-id/resourceGroups/your-rg/providers/Microsoft.CognitiveServices/accounts/your-prediction-resource"

# Initialize clients
training_credentials = ApiKeyCredentials(in_headers={"Training-key": TRAINING_KEY})
trainer = CustomVisionTrainingClient(ENDPOINT, training_credentials)

prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

print("✅ Custom Vision clients initialized")

## Quick Setup: Create & Train Wildlife Project

In [ ]:
def create_wildlife_project():
    """Create a new Custom Vision project for wildlife classification"""
    
    project_name = "Wildlife Camera Trap Classifier"
    description = "Classify animals from camera trap images for ecological research"
    
    print(f"Creating project: {project_name}")
    project = trainer.create_project(project_name, description=description)
    
    print(f"✅ Project created with ID: {project.id}")
    return project

def create_wildlife_tags(project_id):
    """Create classification tags for common wildlife species"""
    
    # Common camera trap categories for quick demo
    categories = [
        "deer", "fox", "bear", "raccoon", "coyote", 
        "rabbit", "squirrel", "bird", "empty", "human"
    ]
    
    tags = {}
    for category in categories:
        tag = trainer.create_tag(project_id, category)
        tags[category] = tag
        print(f"Created tag: {category}")
    
    return tags

# Create project and tags
project = create_wildlife_project()
tags = create_wildlife_tags(project.id)

## Option A: Quick Demo with Computer Vision

**For immediate demonstration (no training data required):**

In [ ]:
def demo_with_computer_vision():
    """
    Use Azure Computer Vision for immediate wildlife detection demo
    Demonstrates the API workflow without needing trained Custom Vision model
    """
    from azure.cognitiveservices.vision.computervision import ComputerVisionClient
    from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
    
    # Computer Vision credentials (same endpoint, different key)
    CV_KEY = "your_computer_vision_key"
    cv_client = ComputerVisionClient(ENDPOINT, ApiKeyCredentials(CV_KEY))
    
    def analyze_wildlife_image(image_url):
        """Analyze image for animals using Computer Vision"""
        
        # Analyze image for objects and animals
        analysis = cv_client.analyze_image(
            image_url, 
            visual_features=[VisualFeatureTypes.objects, VisualFeatureTypes.tags, VisualFeatureTypes.description]
        )
        
        # Extract animal-related detections
        wildlife_detections = []
        
        # Check objects for animals
        for obj in analysis.objects:
            if any(animal in obj.object_property.lower() for animal in ['animal', 'dog', 'cat', 'bird', 'deer', 'bear']):
                wildlife_detections.append({
                    'type': 'object',
                    'name': obj.object_property,
                    'confidence': obj.confidence,
                    'bbox': [obj.rectangle.x, obj.rectangle.y, obj.rectangle.w, obj.rectangle.h]
                })
        
        # Check tags for animal keywords
        animal_tags = []
        animal_keywords = ['deer', 'fox', 'bear', 'raccoon', 'bird', 'cat', 'dog', 'wildlife', 'animal']
        
        for tag in analysis.tags:
            if any(keyword in tag.name.lower() for keyword in animal_keywords):
                animal_tags.append({
                    'name': tag.name,
                    'confidence': tag.confidence
                })
        
        return {
            'description': analysis.description.captions[0].text if analysis.description.captions else "No description",
            'wildlife_objects': wildlife_detections,
            'animal_tags': sorted(animal_tags, key=lambda x: x['confidence'], reverse=True)[:5]
        }
    
    return analyze_wildlife_image

# Initialize Computer Vision demo
cv_demo = demo_with_computer_vision()

In [ ]:
def display_cv_demo_results(image_url):
    """Display Computer Vision analysis results"""
    
    print("🔍 Analyzing image with Azure Computer Vision...")
    result = cv_demo(image_url)
    
    # Display image
    response = requests.get(image_url)
    img = Image.open(io.BytesIO(response.content))
    
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title('Camera Trap Image')
    
    # Display results
    plt.subplot(1, 2, 2)
    if result['animal_tags']:
        tags = [tag['name'] for tag in result['animal_tags'][:5]]
        confidences = [tag['confidence'] for tag in result['animal_tags'][:5]]
        
        plt.barh(tags, confidences, color='steelblue')
        plt.xlabel('Confidence Score')
        plt.title('Animal Detection Tags')
        plt.xlim(0, 1)
    else:
        plt.text(0.5, 0.5, 'No animals detected', ha='center', va='center')
    
    plt.tight_layout()
    plt.show()
    
    print(f"📝 Description: {result['description']}")
    print(f"🦌 Wildlife Objects: {len(result['wildlife_objects'])}")
    print(f"🏷️ Animal Tags: {[tag['name'] for tag in result['animal_tags'][:3]]}")

# Demo with sample wildlife image
sample_url = "https://example.com/wildlife_image.jpg"  # Replace with actual wildlife image URL
# display_cv_demo_results(sample_url)

## Option B: Train Custom Vision Model (Real Implementation)

In [ ]:
def upload_training_images_from_urls(project_id, tags, image_data):
    """
    Upload training images from URLs to Custom Vision
    
    Args:
        project_id: Custom Vision project ID
        tags: Dictionary of tag objects
        image_data: Dictionary with format {tag_name: [list_of_image_urls]}
    """
    
    for tag_name, image_urls in image_data.items():
        if tag_name not in tags:
            print(f"⚠️ Skipping unknown tag: {tag_name}")
            continue
            
        print(f"📤 Uploading {len(image_urls)} images for '{tag_name}'...")
        
        # Process in batches (Custom Vision API limit: 64 images per batch)
        batch_size = 10
        for i in range(0, len(image_urls), batch_size):
            batch_urls = image_urls[i:i+batch_size]
            
            image_list = []
            for url in batch_urls:
                try:
                    image_list.append(ImageFileCreateEntry(
                        name=f"{tag_name}_{len(image_list)}.jpg",
                        contents=requests.get(url).content,
                        tag_ids=[tags[tag_name].id]
                    ))
                except Exception as e:
                    print(f"❌ Failed to download {url}: {e}")
            
            if image_list:
                try:
                    upload_result = trainer.create_images_from_files(
                        project_id, ImageFileCreateBatch(images=image_list)
                    )
                    
                    if upload_result.is_batch_successful:
                        print(f"✅ Uploaded batch {i//batch_size + 1} for {tag_name}")
                    else:
                        print(f"⚠️ Some images failed for {tag_name}")
                        
                except Exception as e:
                    print(f"❌ Upload failed for {tag_name}: {e}")

In [ ]:
# Sample training data (replace with real wildlife image URLs)
training_data = {
    "deer": [
        "https://example.com/deer1.jpg",
        "https://example.com/deer2.jpg",
        # Add 3-8 more deer images
    ],
    "fox": [
        "https://example.com/fox1.jpg", 
        "https://example.com/fox2.jpg",
        # Add 3-8 more fox images
    ],
    "empty": [
        "https://example.com/empty1.jpg",
        "https://example.com/empty2.jpg",
        # Add 3-8 more empty camera trap images
    ]
}

def train_custom_vision_model(project_id):
    """Train the Custom Vision model"""
    
    print("🚀 Starting model training...")
    iteration = trainer.train_project(project_id)
    
    # Wait for training to complete
    while iteration.status != "Completed":
        iteration = trainer.get_iteration(project_id, iteration.id)
        print(f"⏳ Training status: {iteration.status}")
        time.sleep(10)
    
    print("✅ Training completed!")
    
    # Publish the iteration
    publish_iteration_name = "wildlifeClassifier"
    trainer.publish_iteration(project_id, iteration.id, publish_iteration_name, PREDICTION_RESOURCE_ID)
    print(f"📦 Model published as: {publish_iteration_name}")
    
    return iteration, publish_iteration_name

# Upload training data and train model
# upload_training_images_from_urls(project.id, tags, training_data)
# iteration, model_name = train_custom_vision_model(project.id)

## Using Custom Vision for Prediction

In [ ]:
def classify_wildlife_custom_vision(image_source, project_id, model_name="wildlifeClassifier"):
    """
    Classify wildlife using trained Custom Vision model
    
    Args:
        image_source: Image URL or file path
        project_id: Custom Vision project ID  
        model_name: Published model name
    
    Returns:
        Classification results
    """
    
    # Load image
    if image_source.startswith('http'):
        image_data = requests.get(image_source).content
    else:
        with open(image_source, 'rb') as f:
            image_data = f.read()
    
    # Make prediction
    try:
        results = predictor.classify_image(project_id, model_name, image_data)
        
        predictions = []
        for prediction in results.predictions:
            predictions.append({
                'species': prediction.tag_name,
                'confidence': round(prediction.probability, 3),
                'percentage': f"{prediction.probability * 100:.1f}%"
            })
        
        # Sort by confidence
        predictions = sorted(predictions, key=lambda x: x['confidence'], reverse=True)
        
        return {
            'image_source': image_source,
            'predictions': predictions,
            'top_prediction': predictions[0] if predictions else None,
            'high_confidence': predictions[0]['confidence'] > 0.7 if predictions else False
        }
        
    except Exception as e:
        print(f"❌ Prediction error: {e}")
        return None

In [ ]:
def display_custom_vision_results(image_source, result):
    """Display Custom Vision classification results"""
    
    if not result:
        print("❌ No results to display")
        return
    
    # Load image
    if image_source.startswith('http'):
        response = requests.get(image_source)
        img = Image.open(io.BytesIO(response.content))
    else:
        img = Image.open(image_source)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Show image
    axes[0].imshow(img)
    axes[0].axis('off')
    axes[0].set_title('Camera Trap Image', fontsize=14)
    
    # Show predictions
    if result['predictions']:
        species = [p['species'].replace('_', ' ').title() for p in result['predictions'][:5]]
        confidences = [p['confidence'] for p in result['predictions'][:5]]
        
        bars = axes[1].barh(range(len(species)), confidences, color='forestgreen')
        axes[1].set_yticks(range(len(species)))
        axes[1].set_yticklabels(species)
        axes[1].set_xlabel('Confidence Score')
        axes[1].set_title('Species Predictions')
        axes[1].set_xlim(0, 1)
        
        # Add confidence values
        for i, (bar, conf) in enumerate(zip(bars, confidences)):
            axes[1].text(conf + 0.01, i, f'{conf:.3f}', va='center')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    if result['top_prediction']:
        top = result['top_prediction']
        confidence_level = "High" if top['confidence'] > 0.7 else "Medium" if top['confidence'] > 0.4 else "Low"
        print(f"🎯 Best Match: {top['species'].replace('_', ' ').title()} ({top['percentage']}) - {confidence_level} Confidence")

# Example usage (after model is trained and published)
# result = classify_wildlife_custom_vision("test_image.jpg", project.id, "wildlifeClassifier")
# display_custom_vision_results("test_image.jpg", result)

## Batch Processing for Research

In [ ]:
def batch_process_with_custom_vision(image_list, project_id, model_name, survey_name="Wildlife Survey"):
    """Process multiple images with Custom Vision for research analysis"""
    
    results = []
    
    print(f"🔍 Processing {len(image_list)} images with Custom Vision...")
    
    for i, image_path in enumerate(image_list):
        print(f"Processing {i+1}/{len(image_list)}: {image_path.split('/')[-1]}")
        
        result = classify_wildlife_custom_vision(image_path, project_id, model_name)
        
        if result and result['predictions']:
            top_prediction = result['top_prediction']
            results.append({
                'image': image_path.split('/')[-1],
                'species': top_prediction['species'],
                'confidence': top_prediction['confidence'],
                'high_confidence': result['high_confidence'],
                'model_used': 'Azure Custom Vision'
            })
        else:
            results.append({
                'image': image_path.split('/')[-1],
                'species': 'unknown',
                'confidence': 0.0,
                'high_confidence': False,
                'model_used': 'Azure Custom Vision'
            })
    
    return pd.DataFrame(results)

# Example batch processing
# image_paths = ["image1.jpg", "image2.jpg", "image3.jpg"]
# batch_results = batch_process_with_custom_vision(image_paths, project.id, "wildlifeClassifier")

## Research Analysis Dashboard

In [ ]:
def analyze_custom_vision_survey(results_df):
    """Generate research insights from Custom Vision results"""
    
    # Filter wildlife detections
    wildlife_df = results_df[results_df['species'] != 'unknown']
    
    print("=== Custom Vision Wildlife Survey Analysis ===")
    print(f"📷 Total Images Processed: {len(results_df)}")
    print(f"🦌 Wildlife Detected: {len(wildlife_df)} ({len(wildlife_df)/len(results_df)*100:.1f}%)")
    print(f"✅ High-Confidence Detections: {len(wildlife_df[wildlife_df['high_confidence']])} ({len(wildlife_df[wildlife_df['high_confidence']])/len(results_df)*100:.1f}%)")
    
    if len(wildlife_df) > 0:
        species_counts = wildlife_df['species'].value_counts()
        print(f"\n🥇 Most Common Species: {species_counts.index[0]} ({species_counts.iloc[0]} detections)")
        print(f"📊 Species Diversity: {len(species_counts)} different species")
        
        # Create visualization
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Species distribution
        species_counts.head(6).plot(kind='bar', ax=axes[0,0], color='steelblue')
        axes[0,0].set_title('Species Detection Frequency')
        axes[0,0].set_ylabel('Number of Detections')
        axes[0,0].tick_params(axis='x', rotation=45)
        
        # Confidence distribution
        axes[0,1].hist(wildlife_df['confidence'], bins=15, alpha=0.7, color='forestgreen')
        axes[0,1].set_title('Confidence Score Distribution')
        axes[0,1].set_xlabel('Confidence Score')
        axes[0,1].axvline(0.7, color='red', linestyle='--', label='High Confidence Threshold')
        axes[0,1].legend()
        
        # Detection success by confidence threshold
        thresholds = [0.3, 0.5, 0.7, 0.9]
        success_rates = [len(wildlife_df[wildlife_df['confidence'] >= t]) / len(results_df) for t in thresholds]
        
        axes[1,0].plot(thresholds, success_rates, 'o-', color='orange', linewidth=2)
        axes[1,0].set_title('Detection Rate by Confidence Threshold')
        axes[1,0].set_xlabel('Confidence Threshold')
        axes[1,0].set_ylabel('Detection Rate')
        axes[1,0].grid(True, alpha=0.3)
        
        # Model performance summary
        high_conf_rate = len(wildlife_df[wildlife_df['high_confidence']]) / len(results_df) * 100
        detection_rate = len(wildlife_df) / len(results_df) * 100
        
        metrics = ['Detection Rate', 'High Confidence Rate', 'Species Diversity Score']
        values = [detection_rate, high_conf_rate, len(species_counts) / len(wildlife_df) * 100]
        
        axes[1,1].bar(metrics, values, color=['lightblue', 'lightgreen', 'lightyellow'])
        axes[1,1].set_title('Model Performance Metrics')
        axes[1,1].set_ylabel('Percentage / Score')
        axes[1,1].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
    
    return {
        'total_images': len(results_df),
        'wildlife_detected': len(wildlife_df),
        'detection_rate': len(wildlife_df) / len(results_df),
        'high_confidence_rate': len(wildlife_df[wildlife_df['high_confidence']]) / len(results_df),
        'species_diversity': len(wildlife_df['species'].unique()) if len(wildlife_df) > 0 else 0
    }

# Example analysis
# survey_stats = analyze_custom_vision_survey(batch_results)

## Research Data Export

In [ ]:
def export_custom_vision_results(results_df, project_name="wildlife_study"):
    """Export Custom Vision results for research analysis"""
    
    # Main results CSV
    results_df.to_csv(f"{project_name}_custom_vision_results.csv", index=False)
    
    # High-confidence detections for reliable research
    reliable_df = results_df[results_df['high_confidence'] == True]
    reliable_df.to_csv(f"{project_name}_reliable_detections.csv", index=False)
    
    # Species summary for ecological analysis
    wildlife_df = results_df[results_df['species'] != 'unknown']
    if len(wildlife_df) > 0:
        species_summary = wildlife_df.groupby('species').agg({
            'confidence': ['count', 'mean', 'std', 'min', 'max'],
            'high_confidence': 'sum'
        }).round(4)
        
        species_summary.to_csv(f"{project_name}_species_analysis.csv")
    
    print(f"✅ Custom Vision results exported:")
    print(f"   • {project_name}_custom_vision_results.csv")
    print(f"   • {project_name}_reliable_detections.csv") 
    print(f"   • {project_name}_species_analysis.csv")
    
    print(f"\n📊 Ready for R/Python statistical analysis!")

# Example export
# export_custom_vision_results(batch_results, "my_ecological_study")

## Microsoft Custom Vision Advantages for Research

### Why Choose Custom Vision:
1. **No Coding Required**: Visual training interface in portal
2. **Small Data Friendly**: Works with 5-10 images per species
3. **Fast Training**: Model trains in 2-10 minutes
4. **Research APIs**: Confidence scores, batch processing
5. **Azure Integration**: Connects with other Microsoft AI services
6. **Scalable**: Free tier → enterprise depending on research needs

### Research Workflow:
1. **Collect sample images** (5-10 per species minimum)
2. **Upload & tag** via Custom Vision portal
3. **Train model** (automatic, takes ~5 minutes)
4. **Test & iterate** with more training data
5. **Deploy via API** for batch processing
6. **Export results** for statistical analysis

This approach transforms wildlife monitoring from manual review to automated classification, enabling researchers to process camera trap surveys at scale while maintaining scientific rigor through confidence-based validation.